In [3]:
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import MaxPooling2D,Conv2D,Flatten
from keras.optimizers import SGD
from keras.models import model_from_json

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [5]:
X_train = X_train/255
X_test  = X_test/255
y_train = keras.utils.to_categorical(y_train,10)
y_test  = keras.utils.to_categorical(y_test,10)
X_train = X_train.reshape(60000, 28, 28, 1).astype('float32')
X_test = X_test.reshape(10000, 28, 28, 1).astype('float32')

In [7]:
n_classes = 10
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
model.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1, validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 353s 6ms/step - loss: 0.2411 - acc: 0.9270 - val_loss: 0.0579 - val_acc: 0.9803
Epoch 2/5
60000/60000 [==============================] - 354s 6ms/step - loss: 0.0903 - acc: 0.9732 - val_loss: 0.0384 - val_acc: 0.9875
Epoch 3/5
60000/60000 [==============================] - 543s 9ms/step - loss: 0.0675 - acc: 0.9802 - val_loss: 0.0351 - val_acc: 0.9886
Epoch 4/5
60000/60000 [==============================] - 353s 6ms/step - loss: 0.0545 - acc: 0.9835 - val_loss: 0.0322 - val_acc: 0.9899
Epoch 5/5
60000/60000 [==============================] - 360s 6ms/step - loss: 0.0477 - acc: 0.9852 - val_loss: 0.0395 - val_acc: 0.9876


In [11]:
# Saving the model
model_json = model.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights('model-bw.h5')

import numpy as np
import cv2

video = cv2.VideoCapture(0)
top, right, bottom, left = 10, 100, 560, 590

while True :
    aWeight = 0.5
    _,frame = video.read()
    frame = cv2.flip(frame,1)
    cv2.imshow("video",frame)
    h,w = frame.shape[:2]
    roi = frame[top:bottom, right:left]
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(37,37),0)
    cv2.imshow("gray",gray)
    cv2.imshow("blur",blur)
    
    test, test_image = cv2.threshold(blur, 120, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    cv2.imshow("test_image",test_image)
    
    contours,hierarchy = cv2.findContours(test_image,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    if len(contours) > 0 :
        contour = max(contours,key = cv2.contourArea)
        
        if(cv2.contourArea(contour) > 2000):
            x, y, w, h = cv2.boundingRect(contour)
            newImage = test_image[y:y + h, x:x + w]
    
    newImage = cv2.resize(newImage, (28, 28))
    newImage = np.array(newImage)
    newImage = newImage.flatten()
    newImage = newImage.reshape(1,28,28, 1)
    
    result = model.predict(newImage)
    
    print(result)
    cv2.putText(frame,"predicted-value"+ str(result),(10,380),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)   
    #cv2.putText(img, "Deep Network :  " + str(ans3), (10, 380),
                    #cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)        
    cv2.imshow("frame",frame)
    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == 27: # esc key
        break

        
video.release()
cv2.destroyAllWindows()